In [73]:
%load_ext autoreload

import time
import os
import shutil
import pandas as pd
import sys
from extraction_package import ExtractionPipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Next steps

1. https://platform.openai.com/docs/assistants/tools/file-search
make the updates


In [29]:
# # mvt_zinc_comm = ['zinc', 'lead', 'gallium', 'germanium', 'indium', 'silver', 'lead']
# copper_comm = ['copper', 'rhenium', 'molybedenum', 'tellurium', 'gold']
# filename = 'phase_one_copper_top10percent'
# meta_data = pd.read_csv(f'./metadata/{filename}.csv').fillna('')
# meta_data.head()

,File Name,record_id,Identified Commodities
0,NI 43-101 Technical Report for the Whistler Pr...,02d68d7253a5ba5f137bfa8bc8343698b68c5582f8b405...,"gold, copper, silver"
1,NI 43-101 Technical Report for the Ball Creek ...,023b6fbcbcdc4222c843a68f1125f8a321d7e183e40297...,"gold, silver, copper, molybdenum, lead, zinc"
2,NI 43-101 Technical Report for the Rovina Vall...,026b22dd9cce535c80d578f5598ade2801434de617f98c...,"gold, copper"
3,NI 43-101 Technical Report for the Lorraine-Ja...,023f34a1100eee6182a97bf698484a7265b271402e88b2...,"copper, gold, silver, platinum, palladium, mol..."
4,NI 43-101 Technical Report for the Seel Projec...,02ba97ea4a85794cd73e4dffcc941cbac9765ea638fd35...,"copper, gold, molybdenum, lead, zinc, silver, ..."


In [140]:
def get_json_files(directory):
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist.")
        return []

    record_ids  = []
    for file in os.listdir(directory):
         if file.lower().endswith('.json'):
             record_id, name = file.split("_",1)
             record_ids.append(record_id)
    if not record_ids:
        print(f"No .json files found in the directory {directory}.")
    return record_ids

In [142]:
records = get_json_files(f"./extracted/twelve_month/nickel/")
print(records, len(records))

['021e324a0f4833110e3bb480b95c2c694790d51dfef30104ccfd83d1c50cb0c504', '0274a7fed5b71574fb754cf65e5d6f207d083629f7c65c5c3fd0c21449132f1a81', '02df520db54f33fe064407f23b151cf1d95a7923fc9f04e8f38ed1e509daf24811', '02731dec034b83820f6ac3bdb87ffc7611cf258c9b4995a9ca786b792a97bd75e7', '02b1591cfad6809d1d9ea67ea04b34d2df8b16d5f34e4bba701a7924bab9c3b48d', '02923aa605ba4d9a1236aaf59f032901cee3d4e87e40c4216194ab7dd8dc5652ae', '0219eb5842ff591f85afc9e4525c42d3e1a1e0fc404c53d7ea2d286d13966d9414', '02b17cb5e6723055c1b01cc0c0a8b2d4c959fb400d4fd11f70b2fe5154bcbde73a', '02bc065060c61850a9f2024c9506481e026fb806b316f23a3b750a171eb1c53963', '02b53cffff7ae221efa98fa499f3911bf7ff44204c8c5357b932cce664b3baddfe', '02a06b32ebae6ca2a068eccab47eb081497c4a4fb249bf1775682f39eaaf634eea', '0286f00798ab06b12a7ea158d38279675e279a22fffe68455ab626ebaa4c835943', '02f31af1ff73938ae243a969a408871089966ad0750d04335ae2e9ba63f9a4112b', '02c0f68f9dfad2f5f7f50cdfa32a9a86359a75e02178b570444a8d81ae858c0745', '02d66a456ad2bce636

In [144]:
def get_pdf(directory, needed_files, metadata):
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist.")
        return []
        
    df = pd.read_csv(metadata)
    filenames, commodity_list = [], []
    for file in os.listdir(directory):
         if file.lower().endswith('.pdf'):
             record_id, name = file.split("_",1)
             if record_id in needed_files:
                 commodity = list(df.loc[df['record_id'] == record_id, 'Identified Commodities'].values)
                 commodity_list.append(commodity)
                 filenames.append(file)
                 
    return filenames, commodity_list

In [146]:
filenames, commodity_list = get_pdf("./reports/nickel-copper-PGE/", records, "./metadata/phase_one_nickel_copper_PGE_top10percent.csv")
print(filenames, commodity_list, len(filenames))

['02731dec034b83820f6ac3bdb87ffc7611cf258c9b4995a9ca786b792a97bd75e7_NI 43-101 Technical Report for the Retty Lake Project in North America dated March 2010.pdf', '0219eb5842ff591f85afc9e4525c42d3e1a1e0fc404c53d7ea2d286d13966d9414_NI 43-101 Technical Report for the Sudbury Basin Project in North America dated March 2004.pdf', '02df520db54f33fe064407f23b151cf1d95a7923fc9f04e8f38ed1e509daf24811_NI 43-101 Technical Report for the Sudbury Basin Project in North America dated March 2005.pdf', '029d6f91b6c43799e1538eb3c0aefbebf78e4d4576b5167e09095c68c6457b666a_NI 43-101 Technical Report for the Thunder Bay North Project in North America dated September 2009.pdf', '021e324a0f4833110e3bb480b95c2c694790d51dfef30104ccfd83d1c50cb0c504_NI 43-101 Technical Report for the Escape Lake Project in Canada dated October 2019.pdf', '02b1591cfad6809d1d9ea67ea04b34d2df8b16d5f34e4bba701a7924bab9c3b48d_NI 43-101 Technical Report for the Läntinen Koillismaa project in Finland dated May, 2021.pdf', '02b53cffff7

In [148]:
folder_path = "./reports/nickel-copper-PGE/"
output_path = f"./extracted/twelve_month/nickel/"
total = time.time()
for i, filename in enumerate(filenames):
    t = time.time()
    ExtractionPipeline.run(folder_path, filename, commodity_list[i], output_path)
    print(f'Run for file {filename}: {time.time()-t}')
    
print(f'Run in for loop: {time.time()-total}')

2024-06-28 17:23:13,530 - Assistant - INFO - Created the assistant with Model: gpt-4o
2024-06-28 17:23:25,425 - Assistant - INFO - Created Assistant: asst_ihGqJKea01Gaf6K0c1gvipXz
2024-06-28 17:23:42,148 - Assistant - INFO - Run status: completed
2024-06-28 17:23:42,264 - Assistant - INFO - Run: run_OG4tFomkBuHIPSDzubyfZYAR Thread: thread_dL4DtGdUAoa0n4wlX02c3QmN 
 response: YES 

2024-06-28 17:23:42,264 - Assistant - INFO - Response: YES
2024-06-28 17:23:42,265 - Pipeline - INFO - Working on file: 02731dec034b83820f6ac3bdb87ffc7611cf258c9b4995a9ca786b792a97bd75e7_NI 43-101 Technical Report for the Retty Lake Project in North America dated March 2010.pdf title: NI 43-101 Technical Report for the Retty Lake Project in North America dated March 2010.pdf commodities to extract ['copper, nickel, platinum, palladium'] 

2024-06-28 17:23:42,266 - Pipeline - ERROR - 02731dec034b83820f6ac3bdb87ffc7611cf258c9b4995a9ca786b792a97bd75e7_NI 43-101 Technical Report for the Retty Lake Project in Nort

KeyboardInterrupt: 

In [ ]:
# #multiprocessing test

# # ### 10 is the sweet spot
# t = time.time()

# ExtractionPipeline.document_parallel_extract(
#     folder_path,
#     filenames,
#     url_list,
#     commodity,
#     commodity_sign,
#     output_path
#         )

# print("\n-------------------------------------------------------------------------\n")
# print(f'parallel: {time.time()-t}')